In [70]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm 
from scipy.io import loadmat
from mpl_toolkits.mplot3d import Axes3D
from time import time

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [71]:
"""Load the datasets."""
images = loadmat('images.mat') 
classes = loadmat('classes')

images=images['images'].T  
classes=classes['classes'].T

In [72]:
images2d = np.zeros((images.shape[0], 192, 168))
for i in range(images.shape[0]):
    images2d[i] = images[i].reshape(168, 192).T

In [73]:
#replace images and classes with light_images and light_classes for co
X_train, X_test, y_train, y_test = train_test_split(images2d, classes, test_size=0.3, random_state=0)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1689, 192, 168)
(1689, 1)
(725, 192, 168)
(725, 1)


In [74]:
import cv2
from skimage.transform import resize


def build_gabor_kernels(scales=5, orientations=8, ksize=31, fmax=0.25):
    kernels = []
    for u in range(scales):
        for v in range(orientations):
            theta = v * np.pi / orientations
            lambd = 1 / (fmax / (2 ** (u / 2)))
            kernel = cv2.getGaborKernel((ksize, ksize), sigma=4.0, theta=theta,
                                        lambd=lambd, gamma=0.5, psi=0, ktype=cv2.CV_64F)
            kernels.append(kernel)
    return np.array(kernels)

def extract_gabor_features(image, kernels):
    feats = []
    for kernel in kernels:
        filtered = cv2.filter2D(image, cv2.CV_64F, kernel)
        feats.append(filtered)
    return feats

def downsample_features(feats, factor=8):
    return np.array([cv2.resize(f, (f.shape[1] // factor, f.shape[0] // factor)).flatten()
                     for f in feats]).flatten()

def extract_magnitude_features(image, kernels):
    mag_feats = []
    for kernel in kernels:
        filtered = cv2.filter2D(image, cv2.CV_64F, kernel)
        real = np.real(filtered)
        imag = np.imag(filtered)
        magnitude = np.sqrt(real ** 2 + imag ** 2)
        mag_feats.append(magnitude)
    return downsample_features(mag_feats)

def extract_phase_congruency_features(image, kernels, num_orientations=8, num_scales=5, downsample_factor=8, epsilon=1e-6):
    """
    Extract OGPCI features from a grayscale face image using the standard Gabor kernel bank.

    Parameters:
        image (2D np.ndarray): Grayscale image (e.g. 128x128).
        kernels (list): List of Gabor kernels ordered by scale-major then orientation-minor.
        num_orientations (int): Number of orientations (typically 8).
        num_scales (int): Number of scales (typically 5).
        downsample_factor (int): Factor for downsampling.
        epsilon (float): Small constant to avoid division by zero.

    Returns:
        np.ndarray: 1D vector of concatenated downsampled OGPCIs (augmented phase congruency feature vector).
    """
    h, w = image.shape
    image = image.astype(np.float32)
    
    # Group kernels by orientation (reshape into [num_orientations][num_scales])
    kernel_bank = kernels.reshape((num_scales, num_orientations, 31, 31))
    kernel_bank = kernel_bank.transpose((1, 0, 2, 3))

    ogpcis = []
    
    for v, kernel_set in enumerate(kernel_bank):  # loop over orientations
        A_sum = np.zeros((h, w), dtype=np.float32)
        energy_sum = np.zeros_like(A_sum)

        phi_list = []
        A_list = []

        # Step 1: Compute magnitude and phase for each scale at orientation v
        for u in range(num_scales):
            kernel = kernel_set[u]
            response = cv2.filter2D(image, cv2.CV_64F, kernel)
            real = np.real(response)
            imag = np.imag(response)

            A = np.sqrt(real**2 + imag**2)
            phi = np.arctan2(imag, real)

            A_list.append(A)
            phi_list.append(phi)
            A_sum += A

        # Step 2: Compute mean phase at this orientation
        sin_sum = np.sum([A_list[i] * np.sin(phi_list[i]) for i in range(num_scales)], axis=0)
        cos_sum = np.sum([A_list[i] * np.cos(phi_list[i]) for i in range(num_scales)], axis=0)
        phi_mean = np.arctan2(sin_sum + epsilon, cos_sum + epsilon)

        # Step 3: Compute oriented Gabor phase congruency image (OGPCI)
        for u in range(num_scales):
            delta_phi = np.cos(phi_list[u] - phi_mean) - np.abs(np.sin(phi_list[u] - phi_mean))
            energy_sum += A_list[u] * delta_phi

        ogpci = energy_sum / (A_sum + epsilon)

        # Step 4: Downsample OGPCI
        ogpci_down = cv2.resize(ogpci, (w // downsample_factor, h // downsample_factor)).flatten()
        ogpcis.append(ogpci_down)

    # Step 5: Concatenate downsampled OGPCIs
    return np.concatenate(ogpcis)


kernels = build_gabor_kernels()
X_train_gabor = [extract_phase_congruency_features(image=img, kernels=kernels) for img in X_train]
X_test_gabor = [extract_phase_congruency_features(image=img, kernels=kernels) for img in X_test]

In [77]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train_gabor)
X_test_std  = scaler.transform(X_test_gabor)

In [78]:
from sklearn.pipeline import Pipeline
from sklearn import decomposition
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score

def evaluate_pca(X_train, y_train, X_test, y_test, num_components):
    pca = decomposition.PCA(n_components=num_components, whiten=True)
    pca.fit(X_train)
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)

    clf = LogisticRegression(max_iter=2000, fit_intercept=True)
    clf.fit(X_train_pca, y_train.ravel())

    y_test_pred = clf.predict(X_test_pca)

    print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
    print("Test f1 score:", metrics.f1_score(y_test, y_test_pred, average='weighted'))
    print(metrics.classification_report(y_test, y_test_pred))

In [79]:
evaluate_pca(X_train_std, y_train, X_test_std, y_test, num_components=38)

Test Accuracy: 0.9862068965517241
Test f1 score: 0.9863874246033371
              precision    recall  f1-score   support

           1       1.00      0.96      0.98        23
           2       1.00      1.00      1.00        13
           3       1.00      1.00      1.00        18
           4       0.95      1.00      0.98        20
           5       0.96      1.00      0.98        23
           6       1.00      1.00      1.00        11
           7       0.95      0.90      0.93        21
           8       1.00      1.00      1.00        22
           9       1.00      0.96      0.98        24
          10       1.00      0.89      0.94        18
          11       1.00      0.94      0.97        18
          12       1.00      1.00      1.00        22
          13       1.00      1.00      1.00        18
          14       1.00      1.00      1.00        14
          15       1.00      0.96      0.98        24
          16       0.93      1.00      0.96        25
          17 

In [80]:
evaluate_pca(X_train_std, y_train, X_test_std, y_test, num_components=100)

Test Accuracy: 0.9986206896551724
Test f1 score: 0.9986131162284335
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        23
           2       1.00      1.00      1.00        13
           3       1.00      1.00      1.00        18
           4       1.00      1.00      1.00        20
           5       0.96      1.00      0.98        23
           6       1.00      1.00      1.00        11
           7       1.00      1.00      1.00        21
           8       1.00      1.00      1.00        22
           9       1.00      1.00      1.00        24
          10       1.00      1.00      1.00        18
          11       1.00      1.00      1.00        18
          12       1.00      1.00      1.00        22
          13       1.00      1.00      1.00        18
          14       1.00      1.00      1.00        14
          15       1.00      1.00      1.00        24
          16       1.00      1.00      1.00        25
          17 

In [81]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

def evaluate_lda(X_train, y_train, X_test, y_test):
    n_classes = len(np.unique(y_train))
    n_samples, n_features = X_train.shape
    pca = decomposition.PCA(n_components=n_samples - n_classes)
    pca.fit(X_train)
    X_train_pca = pca.transform(X_train)
    W_pca = pca.components_.T

    mean_total = np.mean(X_train_pca, axis=0)
    S_W = np.zeros((X_train_pca.shape[1], X_train_pca.shape[1]))
    S_B = np.zeros_like(S_W)

    for cls in np.unique(y_train):
        X_cls = X_train_pca[(y_train==cls).flatten(),:]
        mean_cls = np.mean(X_cls, axis=0)
        S_W += (X_cls - mean_cls).T @ (X_cls - mean_cls)
        n_cls = X_cls.shape[0]
        diff = (mean_cls - mean_total).reshape(-1, 1)
        S_B += n_cls * (diff @ diff.T)

    eigvals, eigvecs = np.linalg.eig(np.linalg.pinv(S_W) @ S_B)
    idx = np.argsort(-eigvals.real)
    eigvecs = eigvecs[:, idx]
    W_fld = eigvecs[:, :n_classes - 1].real
    W_fisher = W_pca @ W_fld

    X_train_fisher = X_train @ W_fisher
    X_test_fisher  = X_test @ W_fisher

    clf = LogisticRegression(max_iter=2000, fit_intercept=True)
    clf.fit(X_train_fisher, y_train.ravel())

    y_test_pred = clf.predict(X_test_fisher)

    print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
    print("Test f1 score:", metrics.f1_score(y_test, y_test_pred, average='weighted'))
    print(metrics.classification_report(y_test, y_test_pred))

In [82]:
evaluate_lda(X_train_std, y_train, X_test_std, y_test)

Test Accuracy: 0.9972413793103448
Test f1 score: 0.9972416652018101
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        23
           2       1.00      1.00      1.00        13
           3       1.00      1.00      1.00        18
           4       1.00      0.95      0.97        20
           5       0.96      1.00      0.98        23
           6       1.00      1.00      1.00        11
           7       1.00      1.00      1.00        21
           8       1.00      1.00      1.00        22
           9       1.00      1.00      1.00        24
          10       1.00      1.00      1.00        18
          11       1.00      1.00      1.00        18
          12       1.00      1.00      1.00        22
          13       1.00      1.00      1.00        18
          14       1.00      1.00      1.00        14
          15       1.00      1.00      1.00        24
          16       1.00      1.00      1.00        25
          17 